In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import time
import datetime

import pandas as pd
def ends(df, x=5):
    return df.head(x).append(df.tail(x))
setattr(pd.DataFrame,'ends',ends)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import collections

from sklearn.model_selection import KFold

# What we're doing

For many years now, I've been very interested in two related, but different questions:
1. How to _apply_ Machine Learning to Economics by integrating ML into the usual econometric pipeline?
2. What do the ML algorithms themselves teach us about how economies work.

Recently, however, there seems to be a wave of interest (especially coming from the work coming from Stanford University) in these questions. This notebook is about the point 1. In particular, about the problem of estimating causal inference. (Unfortunately, point 2 seems to be still far from being considered.)

We are going to replicate the methodology (and hopefully results) of a recent paper in which the problem of causal inference is seen from the lens of the problem of ''matrix completion'' from the machine learning literature. I found out about this paper thanks to my dear friend Sid Ravinutala.

Since I became aware of this paper, I've become increasingly interested in **Susan Athey**'s work, who's precisely preaching about the importance of using ML in economics. Let's see what she did here...

# Set up of the problem

The paper we want to reproduce is: 

''Matrix Completion Methods for Causal Panel Data Models''
by Susan Athey, Mohsen Bayati, Nikolay Doudchenko, Guido Imbens, Khashayar Khosravi
(NBER Working Paper No. 25132, also available in [arxiv](https://arxiv.org/pdf/1710.10251.pdf))

NOTE: Check [her GitHub](https://github.com/susanathey/), as she made available her code in R. I'll try to implement her functions in Python instead.

The setup of the problem is to consider $N$ units across $T$ time steps. For each unit and time, there are two potential outcomes: $Y_{it}(0)$ if not treated, $Y_{it}(1)$ if treated. The treatment variable can be denoted by $W_{it}$. Hence, the realized outcome is $Y_{it}=Y_{it}(W_{it})$.

The basic goal of causal identification is to estimate 
$$\widehat{ATE} \equiv \frac{1}{NT}\sum_{i,t}\left(Y_{it}(1) - Y_{it}(0)\right.)$$ 
The fundamental problem in causal identification, however, is that we never observe both $Y_{it}(0)$ and $Y_{it}(1)$. We either observe one, or the other.

Athey's et al. (2018) solution is based on the following: First, $Y_{it}(0)$ (or $Y_{it}(1)$) is a $N\times T$ matrix with missing values. Second, there are several methodologies to impute missing values in matrices. Hence, to get the ''counterfactual'' matrix, we can make use of the methods to impute missing values.

Athey et al. focus specifically on imputing missing values in $Y_{it}(0)$ (i.e., the counterfactuals of those treated).

Many algorithms for imputing missing values in matrices are based on matrix factorization methods. The different methods differ in the constraints. See [Udell et al. (2015)](https://arxiv.org/pdf/1410.0342.pdf) for a good review.

Athey's et al. (2018) use the method ''Nuclear Norm Matrix Completion Estimator''.

# Nuclear Norm Matrix Completion Estimator

Let us represent the $N\times T$ matrix we are interested in with $\mathbf{Y}=\mathbf{L^*}+\boldsymbol{\varepsilon}$, where $\boldsymbol{\varepsilon}$ can be considered as measurement error. Athey et al. set up the methodological problem as estimating
$$
\widehat{\mathbf{L}}=\mathrm{arg min}_{\mathbf{L}}\left\{\frac{1}{\left|\mathcal{O}\right|}\left\| \mathbf{P}_{\mathcal{O}}(\mathbf{Y}-\mathbf{L}) \right\|_{F}^2 + \lambda \left\| \mathbf{L} \right\|_{*} \right\}.
$$

- The set of _non-missing_ elements is defined by all $(i,t)\in\mathcal{O}$, while the set of _missing_ elements by  $(i,t)\in\mathcal{M}$.
- The term $\mathbf{P}_{\mathcal{O}}(\mathbf{A})$ is an operator that sets to $0$ all the elements in matrix $\mathbf{A}$ which do not belong to the set of matrix elements $\mathcal{O}$. In other words, we just keep the non-missing elements $\mathcal{O}$, but we replace the missing elements with 0's. $\mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{A})$ would do the opposite.
- The term $\left\| \cdot \right\|_{F}^2$ is the Frobenius Norm: $\left\| \mathbf{A} \right\|_{F}^2 = \sum_{i,j}A_{i,j}^2=\sum_k\sigma_k(\mathbf{A})^2$, where $\sigma_i(\mathbf{A})$ are the singular values of the matrix (given by a Singular Value Decomposition).
- The term $\left\| \cdot \right\|_{*}$ is the **Nuclear Norm** regularization of the minimization problem, given by $\left\| \mathbf{A} \right\|_{*} = \sum_k\sigma_k(\mathbf{A})$. Recall that the singular values are always non-negative.

The regularization has a very important role in the problem, since we want $\mathbf{L}$ to approximate the matrix $\mathbf{Y}$, not make it equal, but only taking into account the information from the non-missing elements. The Nuclear Norm, from other matrix norms, is appropriate here because it makes the problem a convex optimization problem (see discussion in the paper, page. 14).

# Estimation procedure

Athey et al. propose an iterative algorithm which may seem weird at first as a solution to the problem. But before we go there, notice the following:
- In principle, for a complete matrix $\mathbf{Y}$, it holds that $\mathbf{Y} = \mathbf{P}_{\mathcal{O}}(\mathbf{Y}) + \mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{Y})$.
- However, we do not have the values for the opperation $\mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{Y})$ to work. 
- Assuming that we had some $\mathbf{L}_k$ that approximates the missing values of $\mathbf{Y}$ we could have  $\mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{L}_k)$ in lieu.
- But we want to use the information in $\mathbf{P}_{\mathcal{O}}(\mathbf{Y})$ to generate the approximation $\mathbf{L}_k$. The way to do this is by the approximation using the singular value decomposition. This is carried by the function $\mathrm{shrink}_\lambda(\mathbf{A})$. 
- To be clear, $\mathrm{shrink}_\lambda(\mathbf{A})$ is simply a function that returns an approximated version of the matrix $\mathbf{A}$ using a singular value decomposition by taking only the singular values larger or equal than $\lambda$. In other words, if $\mathbf{A}=\mathbf{U}\boldsymbol{\Sigma}\mathbf{V}^T$
$$
\mathrm{shrink}_\lambda(\mathbf{A}) = \mathbf{U}\boldsymbol{\Sigma}_{\mathrm{reduced}}\mathbf{V}^T,
$$
where $\boldsymbol{\Sigma}_{\mathrm{reduced}}$ is equal to $\boldsymbol{\Sigma}$ except that all diagonal elements $k$ less than $\sigma_k(\mathbf{A})<\lambda$ have been set to zero.
- Now, if we initialize $\mathbf{L}_1$ with some value, we can have an approximation for $\mathbf{Y}$ because $\mathbf{L}_2 = \mathrm{shrink}_\lambda(\mathbf{P}_{\mathcal{O}}(\mathbf{Y}) + \mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{L}_1))$.
- Maybe if we repeat this many times, it will converge to some reasonable solution such that $\widehat{\mathbf{L}}=\lim_{n\rightarrow\infty}\mathbf{L}_n$.

I suppose they realized of this by thinking backwards. In other words, the solution $\widehat{\mathbf{L}}$ should satisfy the relation, for a given value of $\lambda$: 
$$\widehat{\mathbf{L}} = \mathrm{shrink}_\lambda(\mathbf{P}_{\mathcal{O}}(\mathbf{Y}) + \mathbf{P}_{\mathcal{O}}^{\bot}(\widehat{\mathbf{L}})).$$ This is what's behind the iterative estimation algorithm. 

### The "Matrix-Completion with Nuclear Norm Minimization" estimator (MCNNM)

Formally, they write it like:
$$
\mathbf{L}_{k+1}(\lambda, \mathcal{O})=\mathrm{shrink}_{\frac{\lambda |\mathcal{O}| }{2}}\left\{\mathbf{P}_{\mathcal{O}}(\mathbf{Y}) + \mathbf{P}_{\mathcal{O}}^{\bot}(\mathbf{L}_{k}(\lambda, \mathcal{O}))\right\}.
$$


### Cross-validation

The approximation, and as a consequence the estimate, will depend on the value of $\lambda$. As a hyper-parameter, we will choose its value through cross-validation.

Athey et al. propose to use some sort of $K$-fold cross-validation, and choose $K$ such that 
$$\left|\mathcal{O}_k\right|/\left|\mathcal{O}\right| = \left|\mathcal{O}\right|/(NT).$$
That is, such that the fraction of non-missing values in a given validation set $\mathcal{O}_k$ is equal to the fraction of non-missing values in the original matrix. We will do this making use of Grid Search.

# Python implementation

In [4]:
# Let's begin with the P operators
def PO_operator(Amat, setO):
    """
    Set all elements that are not in setO to 0. It assumes that setO
    comes from applying np.nonzero(A==condition) to a matrix.
    """
    Anew = np.zeros_like(Amat)
    Anew[setO] = Amat[setO]

    return Anew

def POcomp_operator(Amat, setO):
    """
    The complement of PO_operator.
    Set all elements that are in setO to 0. It assumes that setO
    comes from applying np.nonzero(A==condition) to a matrix.
    """
    Anew = np.copy(Amat)
    Anew[setO] = 0

    return Anew

In [5]:
# Lets code the shrink operator
def shrink(Amat, lamb=0, doprint=False):
    """
    This generates a reduced version of A given by the singular value decomposition.
    It only takes the singular above lamb.
    """
    U, Sigma, VT = np.linalg.svd(Amat, full_matrices=False)
    
    if(doprint): print(Sigma)
    
    Sigma[Sigma < lamb] = 0
    
    return U@np.diag(Sigma)@VT

In [6]:
# The loss function
def loss(Ymat, Lmat, setO, doprint=False):
    Ocardinality = len(setO[0])
    diffmat = Ymat - Lmat
    if(doprint): print(diffmat)
    outmat = PO_operator(diffmat, setO)**2
    return (outmat.sum().sum()/Ocardinality)**0.5

In [13]:
# The "Matrix-Completion with Nuclear Norm Minimization" estimator
from sklearn.base import BaseEstimator, TransformerMixin
class MatrixCompletion_NNM(BaseEstimator, TransformerMixin):
    """
    This implements the iterative procedure to estimate L. 
    Since L is really the matrix Y, but with the missing values
    imputed, we chose the Transformer 'Mixin'.

    Parameters
    ----------
    setOk : array-like
        This comes, for example, from applying np.nonzero(A==condition) to a matrix
    
    missing_values : number, string, np.nan (default) or None
        The placeholder for the missing values. All occurrences of
        `missing_values` will be imputed.
        
    lamb : int (default=0)
        This is the regularization parameter, which should be non-negative since
        it is a minimization procedure.
        
    epsilon : float (default=0.001)
        Desired accuracy of the estimation.
    
    max_iters : integer (default=100)
    
    doprint : boolean, optional (default=False)
        Prints different results of the estimation steps.
        
    printbatch : integer, optional if doprint==True (default=10)
        After how many iterations to print the loss function.
        
    copy : boolean, optional, default True
        Set to False to perform inplace row normalization and avoid a
        copy (if the input is already a numpy array).
    

    Attributes
    ----------
    Lest_ : array-like, shape (N, T)
        This is the estimate of L.
    
    loss_ : float
        The root-square of the mean square error of the observed elements.
        
    iters_ : int
        Number of iterations it took the algorithm to get the desired
        precision given by the parameter 'epsilon'.
        
    Examples
    --------
    >>> import numpy as np
    >>> data = np.array([[1,2,np.nan, 0],[np.nan,2,2,1],[3,0,1,3],[0,0,2,np.nan]])
    >>> observedset = np.nonzero(~np.isnan(data))
    >>> my_mcnnm = MatrixCompletion_NNM(setOk=observedset, lamb=2.5, epsilon=10**(-6), doprint=False)
    >>> print(data)
    [[  1.   2.  nan   0.]
    [ nan   2.   2.   1.]
    [  3.   0.   1.   3.]
    [  0.   0.   2.  nan]]
    >>> print(my_mcnnm.fit(data))
    MatrixCompletion_NNM(copy=True, doprint=False, epsilon=1e-06, lamb=2.5,
               max_iters=100, printbatch=10,
               setOk=(array([0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3], dtype=int64), 
               array([0, 1, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2], dtype=int64)))
    >>> print(my_mcnnm.transform(data))
    [[ 0.98082397  2.00915981  3.49022202  0.01813745]
     [ 1.55601129  1.33063721  2.38547663  0.97025289]
     [ 3.00567286  0.33134019  0.80761928  3.00948113]
     [ 0.0419737   0.87320627  1.48553996 -0.39839039]]
    >>> print(my_mcnnm.fit_transform(data))
    [[ 0.98082397  2.00915981  3.49022202  0.01813745]
     [ 1.55601129  1.33063721  2.38547663  0.97025289]
     [ 3.00567286  0.33134019  0.80761928  3.00948113]
     [ 0.0419737   0.87320627  1.48553996 -0.39839039]]
    >>> print(my_mcnnm.transform([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]]))
    [[ 0.98082397  2.00915981  3.49022202  0.01813745]
     [ 1.55601129  1.33063721  2.38547663  0.97025289]
     [ 3.00567286  0.33134019  0.80761928  3.00948113]
     [ 0.0419737   0.87320627  1.48553996 -0.39839039]]
    
    Notes
    -----
    
    
    """
    

    def __init__(self, setOk=None, #missing_values=np.nan, 
                 lamb=0, epsilon=0.001, max_iters=100, 
                 doprint=False, printbatch=10, copy=True):
        self.setOk = setOk 
        #self.missing_values = missing_values 
        self.lamb = lamb 
        self.epsilon = epsilon 
        self.max_iters = max_iters 
        self.doprint = doprint 
        self.printbatch = printbatch
        self.copy = copy

    def fit(self, X, y=None):
        """Fit the estimator to the matrix X (which is 
        really the matrix Y in Athey et al.'s paper).
        
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            The training input samples.
        y : Ignored
            There is no need of a target in a transformer, yet the pipeline API
            requires this parameter.
            
        Returns
        -------
        self : object
            Returns self.
        """
        
        # First, I should check that the missing values are right
        #Ynew = np.zeros_like(self.Ymat)
        #Ynew = self.missing_values
        #Ynew[self.setOk] = self.Ymat[self.setOk]

        assert (len(self.setOk) > 0), "setOk should be a non-empty array"
        assert (self.lamb > 0), "lamb is the lambda parameter which should be larger than zero"

        N, T = X.shape

        # Initialize L to the observed (non-missing) values of Y given by the set setOk
        Lprev = PO_operator(X, self.setOk)

        # Initialization of error with a highvalue and the iteration
        error = N*T*10**3
        iteration = 0

        while((error > self.epsilon) and (iteration < self.max_iters)):
            Lnext = shrink(PO_operator(X, self.setOk) + 
                           POcomp_operator(Lprev, self.setOk), lamb = self.lamb)

            # Updating values
            Lprev = Lnext.copy()
            error = loss(X, Lprev, self.setOk)
            iteration = iteration + 1

            if(self.doprint and (iteration%self.printbatch==0 or iteration==1)):
                print("Iteration {}\t Current loss: {}".format(iteration, error))

        if(self.doprint):
            print("")
            print("Final values:")
            print("Iteration {}\t Current loss: {}".format(iteration, error))
            print("")
            print(X)
            print(np.round(Lnext, 2))
        
        self.iters_ = iteration
        self.loss_ = error
        self.Lest_ = Lnext
        
        # Return the transformer
        return self

    def transform(self, X):
        """ 
        Actually returning the estimated matrix, in which we have 
        imputed the missing values of X (matrix Y).
        
        Parameters
        ----------
        X : {array-like, sparse-matrix}, shape (N, T)
            The input data to complete.
            
        Returns
        -------
        X_transformed : array, shape (N, T)
            The array the completed matrix.
        """
        try:
            getattr(self, "Lest_")
        except AttributeError:
            raise RuntimeError("You must estimate the model before transforming the data!")
        
        return self.Lest_

In [14]:
data = np.array([[1,2,np.nan, 0],[np.nan,2,2,1],[3,0,1,3],[0,0,2,np.nan]])
observedset = np.nonzero(~np.isnan(data))

my_mcnnm = MatrixCompletion_NNM(setOk=observedset, lamb=2.5, epsilon=10**(-6), doprint=False)

#print(my_mcnnm.transform(data))

print(data)

print(my_mcnnm.fit(data))

print(my_mcnnm.transform(data))

print(my_mcnnm.fit_transform(data))

print(my_mcnnm.transform([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]]))

[[  1.   2.  nan   0.]
 [ nan   2.   2.   1.]
 [  3.   0.   1.   3.]
 [  0.   0.   2.  nan]]
MatrixCompletion_NNM(copy=True, doprint=False, epsilon=1e-06, lamb=2.5,
           max_iters=100, printbatch=10,
           setOk=(array([0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3], dtype=int64), array([0, 1, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2], dtype=int64)))
[[ 0.98082397  2.00915981  3.49022202  0.01813745]
 [ 1.55601129  1.33063721  2.38547663  0.97025289]
 [ 3.00567286  0.33134019  0.80761928  3.00948113]
 [ 0.0419737   0.87320627  1.48553996 -0.39839039]]
[[ 0.98082397  2.00915981  3.49022202  0.01813745]
 [ 1.55601129  1.33063721  2.38547663  0.97025289]
 [ 3.00567286  0.33134019  0.80761928  3.00948113]
 [ 0.0419737   0.87320627  1.48553996 -0.39839039]]
[[ 0.98082397  2.00915981  3.49022202  0.01813745]
 [ 1.55601129  1.33063721  2.38547663  0.97025289]
 [ 3.00567286  0.33134019  0.80761928  3.00948113]
 [ 0.0419737   0.87320627  1.48553996 -0.39839039]]


In [15]:
from sklearn.grid_search import GridSearchCV

Y = np.random.logseries(0.3, size = (30, 40))
lambda_values = [0.5*2**n for n in np.arange(0, 5, 0.5)]

print(Y)
print(lambda_values)

[[2 1 1 ..., 1 1 2]
 [2 1 1 ..., 1 3 2]
 [1 1 1 ..., 1 1 1]
 ..., 
 [1 1 2 ..., 2 1 1]
 [1 1 1 ..., 1 3 1]
 [1 2 1 ..., 1 1 1]]
[0.5, 0.70710678118654757, 1.0, 1.4142135623730951, 2.0, 2.8284271247461903, 4.0, 5.6568542494923806, 8.0, 11.313708498984761]


C:\Users\agomez\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\agomez\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
grid = GridSearchCV(pipe, cv=5, n_jobs=1, param_grid=param_grid)

In [ ]:
# Cross-validation
def do_CV()

# Real Data

I have to say I was slightly disappointed by the application of the method to ''real'' data. Athey et al. do not really apply to actual data, with actual treated units at specific times. In other words, in the paper they use a matrix where there are no missing entries (i.e., there is no treatment), and they choose a subset of units and some randomly selected times to assign a hypothetical treatment (that is kept following the initial period). After ''pretending'' the data is missing for those units at those times, the aim is the reconstruct the missing entries in the matrix.

They compare the reconstruction error for different methods, such as Difference-in-differences, and different versions of synthetic control approaches, which Athey et al. show can all be interpreted under the unifying framework of matrix factorization methods:
$$
    \widehat{\mathbf{L}}^{\mathrm{est}}=\mathbf{A}^{\mathrm{est}}{\mathbf{B}^{\mathrm{est}}}^T,
$$
for some $\mathbf{A}^{\mathrm{est}}$ and $\mathbf{B}^{\mathrm{est}}$ which are solutions to optimization procedures of the form
$$
    \mathrm{arg min}_{(\mathbf{A},\mathbf{B})}\left\{\frac{1}{\left|\mathcal{O}\right|}\left\| \mathbf{P}_{\mathcal{O}}(\mathbf{Y}-\mathbf{A}\mathbf{B}^T) \right\|_{F}^2 + \mathrm{regularization~~on~~} \mathbf{A}\mathrm{~~and~~}\mathbf{B}\right\},
$$
plus some other restrictions on $\mathbf{A}$ and $\mathbf{B}$.

Because they artificially create the ''treatment'', we are going to implement their two options:
1. **Simultaneous adoption**: where $N_t$ units adopt simultaneously the treatment in period $t_0+1$, while the other units never adopt the treatment.
2. **Staggered adoption**: where $N_t$ units adopt the treatment, but each does so in randomly chosen times.

In [ ]:
def simul_adopt(Ymat, treated_units, t0):
    """
    The function assumes that t0 are integer values that go
    from 1 to T, where T is the wide length of the matrix Ymat.
    """
    Ynew = Ymat.copy()
    Ynew[treated_units, t0:] = np.nan
    return Ynew

def stag_adopt(Ymat, treated_units, t0):
    """
    Here we assume that units get treated at times 
    t after t0, chosen uniformly at random from the 
    remaining times.
    """
    N, T = Ymat.shape
    ts = np.random.choice(np.arange(t0,T), size = len(treated_units))
    Ynew = Ymat.copy()
    for i, unit in enumerate(treated_units):
        Ynew[unit,ts[i]:] = np.nan
    return Ynew
    

In [ ]:
simul_adopt(Lest, [0,2], 2)

In [ ]:
stag_adopt(Lest, [0,2], 1)

## Using the Abadie-Diamond-Hainmueller California Smoking Data

In [ ]:
PATH_FROM_URL = "https://raw.githubusercontent.com/susanathey/MCPanel/master/tests/examples_from_paper/california/"

X = pd.read_csv(PATH_FROM_URL + 'smok_covariates.csv',header=None).values
Y = pd.read_csv(PATH_FROM_URL + "smok_outcome.csv", header=None).values.T
treat = pd.read_csv(PATH_FROM_URL + 'smok_treatment.csv',header=None).values.T
years = np.arange(1970,2001)

## First row (treated unit)
CA_y = Y[1,:]

## Working with the rest of matrix (i.e., dropping first row)
treat = treat[1:,:] 
Y = Y[1:,:]


In [ ]:
# Note that np.printoptions() is only available in version 1.15 and later of Numpy.
with np.printoptions(precision=2, suppress=True):
    print(X)
    print(X.shape)
    print(Y)
    print(Y.shape)
    print(treat)
    print(treat.shape)

In [ ]:
plt.imshow(Y, cmap=plt.cm.gray);